In [68]:
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.stats import ttest_ind

## task 2

In [104]:
sales_df = pd.read_csv('2022-04-01T12_df_sales.csv')
# sales_df['date'] = sales_df['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

In [105]:
start_dt = datetime(2022, 3, 16)
end_dt = datetime(2022, 3, 23)

In [106]:
sales_df['date'] = pd.to_datetime(sales_df['date'])

In [107]:
sales_df = sales_df[(sales_df['date'] < end_dt) & (sales_df['date'] >= start_dt)]


In [108]:
sales_df_agg = sales_df[['user_id', 'price']].groupby('user_id')['price'].sum().reset_index()

In [109]:
sales_df_agg['user_id'].nunique()

24600

In [110]:
split_df = pd.read_csv('experiment_users.csv')

In [111]:
split_df.groupby('pilot')['user_id'].nunique()

pilot
0    11769
1    11564
Name: user_id, dtype: int64

In [115]:
df = pd.merge(split_df, sales_df_agg, how='left', on='user_id').fillna(0)

In [116]:
df = df.rename({'price': 'revenue'}, axis=1)

In [117]:
df.groupby('pilot').user_id.nunique()

pilot
0    11769
1    11564
Name: user_id, dtype: int64

In [118]:
ttest_ind(df[df['pilot'] == 1]['revenue'], df[df['pilot'] == 0]['revenue'])

TtestResult(statistic=1.2837567415000515, pvalue=0.19923983306424942, df=23331.0)

# task 3

In [119]:
sales_df = pd.read_csv('2022-04-01T12_df_sales.csv')

In [135]:
users_sample = sales_df.groupby('user_id')['sale_id'].count().reset_index().rename({'sale_id': 'sales_cnt'}, axis=1)
users_sample = users_sample[users_sample['sales_cnt'] > 1]

In [137]:
users_sample.head()

,user_id,sales_cnt
0,000096,2
1,0000d4,2
2,0000de,3
3,0000e4,2
6,0001e2,2


In [160]:
users_df = sales_df.merge(users_sample, how='inner', on='user_id')[['date', 'user_id']]
users_df['date'] = pd.to_datetime(users_df['date'])
users_df = users_df.sort_values(['user_id', 'date'])

In [161]:
users_df.head(10)

,date,user_id
149010,2022-03-04 11:15:55,000096
149011,2022-03-22 13:16:09,000096
141031,2022-02-28 16:32:09,0000d4
141032,2022-03-27 11:26:30,0000d4
65107,2022-02-11 18:57:15,0000de
65108,2022-03-11 19:33:20,0000de
65109,2022-03-25 17:01:47,0000de
140100,2022-02-28 12:41:47,0000e4
140101,2022-03-27 14:54:35,0000e4
167837,2022-03-25 11:01:07,0001e2


In [162]:
users_df['shift'] = users_df.groupby('user_id')['date'].shift()

In [164]:
users_df = users_df[users_df['shift'].isnull() == False]

In [166]:
users_df['delta'] = users_df['date'] - users_df['shift']

In [168]:
users_df.delta.mean()

Timedelta('17 days 07:37:57.653431857')

## task 4

In [175]:
import numpy as np
from scipy import stats


def get_ttest_pvalue(metrics_a_group, metrics_b_group):
    return stats.ttest_ind(metrics_a_group, metrics_b_group).pvalue

In [176]:
metrics_a_group = np.array([964, 1123, 962, 1213, 914, 906, 951, 1033, 987, 1082])
metrics_b_group = np.array([952, 1064, 1091, 1079, 1158, 921, 1161, 1064, 819, 1065])
pvalue = get_ttest_pvalue(metrics_a_group, metrics_b_group)
# pvalue = 0.6122191629541949

In [177]:
print(pvalue)

0.6122191629541949
